In [1]:
# Imports
import pandas as pd
import re


In [2]:
df_cpi = pd.read_csv("data/cpi.csv")
df_interest = pd.read_csv("data/interest.csv")
df_rent_index = pd.read_csv("data/rentIndex.csv")
df_vacant = pd.read_csv("data/vacant.csv")
df_properties = pd.read_csv("data/properties.csv")
df_geo_attributes = pd.read_csv("data/geo_attributes.csv")
df_test = pd.read_csv("data/test.csv")
df_train = pd.read_csv("data/train.csv")


In [3]:
df_cpi.head()


,Data Series,CPI
0,2022 Dec,111.186
1,2022 Nov,110.959
2,2022 Oct,109.893
3,2022 Sep,110.339
4,2022 Aug,109.863


In [4]:
df_interest.head()


,Data Series,InterestRate
0,2022 Dec,1.48
1,2022 Nov,1.47
2,2022 Oct,1.17
3,2022 Sep,1.03
4,2022 Aug,0.95


In [5]:
df_rent_index.head()


,Data Series,RentIndex
0,2022 4Q,148.1
1,2022 3Q,137.9
2,2022 2Q,127.0
3,2022 1Q,119.0
4,2021 4Q,114.2


In [6]:
df_vacant.head()


,Data Series,Available,Vacant
0,2022 4Q,34084,524
1,2022 3Q,34084,514
2,2022 2Q,34084,452
3,2022 1Q,34084,517
4,2021 4Q,34084,576


In [7]:
df_properties.head()


,area,floorRange,propertyType,district,typeOfArea,tenure,street,project,marketSegment,property_key
0,226.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-7b6c69000
1,194.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-320ed2726
2,348.0,06-10,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-2bc5c4951
3,223.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-f060c5be0
4,195.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-a8d1004a9


In [8]:
df_geo_attributes.head()


,street,project,district,lat,lng,num_schools_1km,num_supermarkets_500m,num_mrt_stations_500m
0,LIM AH WOO ROAD,SUITES @ GUILLEMARD,15,1.312520,103.891760,7,2,3
1,HOLLAND ROAD,LOFT@HOLLAND,10,1.312364,103.797169,0,3,4
2,SOUTH BUONA VISTA ROAD,VIVA VISTA,5,1.279803,103.785910,0,0,2
3,UPPER SERANGOON ROAD,PARK RESIDENCES KOVAN,19,1.357934,103.881932,7,2,1
4,NATHAN ROAD,LOFT @ NATHAN,10,1.294461,103.827621,2,4,4


In [9]:
df_train.head()


,property_key,contractDate,price
0,p-c1ce31019,2018-03-01,1850000.0
1,p-a7fd7deb5,2018-03-01,1030000.0
2,p-2a981615e,2018-03-01,4300000.0
3,p-2bb570b5d,2018-03-01,1400888.0
4,p-4ccd6e1db,2018-03-01,725000.0


## Process macro and micro features


### Macro

Make all monthly to join on date

In [10]:
df_cpi["date"] = pd.to_datetime(df_cpi["Data Series"], format="%Y %b ").dt.to_period("M")


In [11]:
df_interest["date"] = pd.to_datetime(df_interest["Data Series"], format="%Y %b ").dt.to_period("M")


In [12]:
df_rent_index["date"] = pd.to_datetime(
    df_rent_index["Data Series"].str.replace(r"(\d+) (\d)Q ", r"\1-Q\2", regex=True)
).dt.to_period("M")
df_rent_index = (
    df_rent_index.set_index("date").resample("M", convention="end").interpolate("linear")
)
df_rent_index = df_rent_index.reset_index()


C:\Users\Dante\AppData\Local\Temp\ipykernel_8576\2552912560.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_rent_index['date'] = pd.to_datetime(df_rent_index['Data Series'].str.replace(r"(\d+) (\d)Q ", r"\1-Q\2", regex=True)).dt.to_period('M')


In [13]:
df_rent_index.head()


,date,Data Series,RentIndex
0,2018-01,2018 1Q,102.800000
1,2018-02,NaN,103.133333
2,2018-03,NaN,103.466667
3,2018-04,2018 2Q,103.800000
4,2018-05,NaN,103.900000


In [14]:
df_vacant["date"] = pd.to_datetime(
    df_vacant["Data Series"].str.replace(r"(\d+) (\d)Q ", r"\1-Q\2", regex=True)
).dt.to_period("M")
df_vacant = df_vacant.set_index("date").resample("M", convention="end").interpolate("linear")
df_vacant = df_vacant.reset_index()


C:\Users\Dante\AppData\Local\Temp\ipykernel_8576\1142357248.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_vacant['date'] = pd.to_datetime(df_vacant['Data Series'].str.replace(r"(\d+) (\d)Q ", r"\1-Q\2", regex=True)).dt.to_period('M')


In [15]:
df_vacant.head()


,date,Data Series,Available,Vacant
0,2018-01,2018 1Q,30153.0,3950.000000
1,2018-02,NaN,30624.0,4135.000000
2,2018-03,NaN,31095.0,4320.000000
3,2018-04,2018 2Q,31566.0,4505.000000
4,2018-05,NaN,31734.0,4048.333333


### Micro

In [16]:
df_properties.head()


,area,floorRange,propertyType,district,typeOfArea,tenure,street,project,marketSegment,property_key
0,226.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-7b6c69000
1,194.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-320ed2726
2,348.0,06-10,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-2bc5c4951
3,223.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-f060c5be0
4,195.0,01-05,Condominium,4,Strata,99 yrs lease commencing from 2007,COVE DRIVE,TURQUOISE,CCR,p-a8d1004a9


In [17]:
df_properties.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31530 entries, 0 to 31529
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   area           31530 non-null  float64
 1   floorRange     31530 non-null  object 
 2   propertyType   31530 non-null  object 
 3   district       31530 non-null  int64  
 4   typeOfArea     31530 non-null  object 
 5   tenure         31530 non-null  object 
 6   street         31530 non-null  object 
 7   project        31530 non-null  object 
 8   marketSegment  31530 non-null  object 
 9   property_key   31530 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 2.4+ MB


In [18]:
df_properties.groupby("floorRange")["property_key"].agg("count")


floorRange
-         4957
01-05    12037
06-10     5693
11-15     3912
16-20     2375
21-25     1052
26-30      662
31-35      411
36-40      200
41-45       94
46-50       45
51-55       32
56-60       22
61-65       18
66-70        5
71-75        1
B1-B5       14
Name: property_key, dtype: int64

In [19]:
df_properties.groupby("propertyType")["property_key"].agg("count")


propertyType
Apartment                 9782
Condominium              14881
Detached                   438
Executive Condominium     1946
Semi-detached             1425
Strata Detached             88
Strata Semi-detached       183
Strata Terrace             470
Terrace                   2317
Name: property_key, dtype: int64

In [20]:
df_properties.groupby("district")["property_key"].agg("count")


district
1      529
2      354
3      852
4      728
5     1228
6        8
7      299
8      544
9     1984
10    2548
11    1394
12    1081
13     723
14    1524
15    3123
16    1565
17     587
18    1348
19    3627
20    1221
21    1126
22     765
23    1675
25     427
26     425
27     866
28     979
Name: property_key, dtype: int64

In [21]:
df_properties.groupby("typeOfArea")["property_key"].agg("count")


typeOfArea
Land       4191
Strata    27339
Name: property_key, dtype: int64

In [22]:
df_properties.groupby("tenure")["property_key"].agg("count")


tenure
100 yrs lease commencing from 1986          11
102 yrs lease commencing from 1978          20
102 yrs lease commencing from 1996          19
103 yrs lease commencing from 1974           4
103 yrs lease commencing from 1975           6
                                         ...  
999999 yrs lease commencing from 1958        1
999999 yrs lease commencing from 1963        1
999999 yrs lease commencing from 1990        1
999999 yrs lease commencing from 1993        1
Freehold                                 14723
Name: property_key, Length: 122, dtype: int64

In [23]:
df_properties.groupby("street")["property_key"].agg("count")


street
ADAM ROAD           21
ADIS ROAD           15
AH HOOD ROAD        28
AH SOO GARDEN        3
AIDA STREET        125
                  ... 
YUK TONG AVENUE      5
YUNNAN CRESCENT     16
ZEHNDER ROAD         4
ZION CLOSE          12
ZION ROAD           15
Name: property_key, Length: 1171, dtype: int64

In [24]:
df_properties.groupby("project")["property_key"].agg("count")


project
# 1 LOFT               8
# 1 SUITES            12
1 CANBERRA            56
1 KING ALBERT PARK    12
1 MOULMEIN RISE        7
                      ..
YUEN SING MANSION      1
YUNNAN GARDENS        16
ZEDGE                 20
ZENITH                15
ZEPHYR PARK            2
Name: property_key, Length: 2753, dtype: int64

In [25]:
df_properties.groupby("marketSegment")["property_key"].agg("count")


marketSegment
CCR     7002
OCR    15737
RCR     8791
Name: property_key, dtype: int64

- floorRange: one hot encode
- propertyType: one hot encode
- district: to join with df_geo_attributes
- typeOfArea: one hot encode
- tenure: set to Freehold or non-Freehold
- street: to join with df_geo_attributes
- project: to join with df_geo_attributes
- marketSegment: one hot encode